# Notes on the Szegedy Quantum Walks | Introduction to random walks

**Table of contents**

0. _Introduction to random walks_
    1. Markov chains and properties
    2. Boltzmann distribution and the Ising model
    4. Metropolis-Hastings algorithm
    5. Python implementation
1. _Introduction to quantum walks_
    1. From Markov chains to unitary quantum walks
    2. Construction of $W$
    3. Python implementation

## Markov chains and properties

A (finite) Markov chain is specified by a state space $\Omega$ and a transition matrix $P \in \mathbb{R}^{|\Omega| \times |\Omega|}$ where 
$$P(x, y) = \mathrm{Pr}[X_{t+1} = y \mid X_t = x]$$ 
is the probability of moving from state $x$ to $y$. Some properties:

* _**column stochasticity**_: each row represents a probability distribution, thus $P_{ij} \ge 0, \sum_i P_{ij} = 1$ for $i, j \in \Omega$. This fixes also the notation used, with $P(x,y) = P_{yx}$ and a distribution $\mu$ being a column-vector, with time evolution $\mu_{t+1} = P \mu_t$.

    (Of course one can use the dual notation in which case the property that needs to hold is the row stochasticity).

* **theorem**: a stationary distribution $\pi$ exists, $\pi = P \pi$, for any stochastic matrix $P$ (finite case only).

* _**irreducibility**_: the directed graph induced by $P$ is strongly connected, or equivalently $\forall x, y \in \Omega \; \exists t \ge 1 \,:\, P^t(x,y) > 0$. This ensures the chain can reach any state.

* **theorem**: the chain is irreducible iff the stationary distribution is unique.

* _**aperiodicity**_: the _period_ of the state $x$ is $d(x) = \mathrm{gcd}\{ t \ge 1 \mid P^t_{xx} > 0 \}$; a markov chain is _aperiodic_ if for all states $x$ we have $d(x) = 1$.

* **theorem**: in a _irreducible_ chain, all states have the same period; thus, if _one_ state has a self-loop $P_{xx} > 0$ then the entire chain is aperiodic. If an irreducible chain is aperiodic too, then it converges to the stationary distribution from any initial distribution.

* _**detailed balance/reversibility**_: the markov chain satisfies the detailed balance with respect to $\pi$ iff $\pi_x P_{yx} = \pi_y P_{xy} \; \forall x,y \in \Omega$. In case, the MC is said to be _reversible_

* **theorem**: if the irreducible $P$ is reversible then there exists a unique stationary distribution.

* **theorem**: if the irreducible $P$ is symmetric then it is reversible by design; not all reversible $P$ are symmetric though.

The performance of a MC is quantified via the mixing time, i.e. how quickly the chain approaches stationarity in the worst initial state. One definition uses the total variation norm, 
$$\lVert \mu \rVert_\text{TV} =\sup_x |\mu_x| $$
Let $\mu_t^{(x)} $ be the distribution at time $t$ starting from the initial state $x$ and $\pi$ the unique stationary distribution. The **$\varepsilon$-mixing time** is 
$$t_\text{mix}(\varepsilon) = \min \{t \mid \max_{x \in \Omega} \lVert \mu_t^{(x)} - \pi \rVert_\text{TV} \le \varepsilon \}$$
For finite irreducible reversible chains, convergence rates are closely controlled by the spectral gap. Let the eigenvalues of $P$ be $1 = \lambda_1 \ge \lambda_2 \ge \cdots \ge \lambda_N \ge -1$. The spectral gap is 
$$\gamma := 1 - \max \{|\lambda_2|, |\lambda_{N}| \}$$

(when the chain is lazy or otherwise has nonnegative spectrum, the simpler form $\gamma := 1 - \lambda_2$ may be used)

Then, $t_\text{mix}(\epsilon) \le \gamma^{-1}$ up to logarithmic terms in $\varepsilon^{-1}$ and $\pi^{-1}$.

## Boltzmann distribution and the Ising model

Let $\Omega$ be a finite configuration space and let the Hamiltonian be a real-valued function $E: \Omega \to \mathbb{R}$ assigning an energy value to each configuration. Let $\beta \ge 0$ be an inverse termperature parameter. The Boltzmann distribution $\mu_\beta$ is 
$$\mu_\beta(x) := \frac{e^{-\beta E(x)}}{Z(\beta)}$$
where $Z(\beta) := \sum_x e^{-\beta E(x)}$ is the partition function. 

The parameter $\beta$ controls the concentration: at high temperatures ($\beta \to 0$) we have $\mu_\beta$ approaching uniform distribution and at low temperatures ($\beta \to \infty$) it concetrates around the ground states of $E$.

A notable example of models we can study are the Ising model. We classify them here by geometry:

The Ising model on a graph $G = (V, E)$ has variables $x_i \in \{ \pm 1 \}$ for $i \in V$ and the energy term can be expressed as
$$E(x) = - \sum_{\{i,j\} \in E} J_{ij} x_i x_j - h_i x_i$$
This means there are 2-body terms only (here the notation is $k=2$ where $k$ refers to the number of interacting particles). A sub-class of these is the lattice Ising which has a further restriction on the graph being a grid with regular structure, this imposes a bounded interaction neighborhood (here the notation is $d$).

The $(k, d)$-local Ising model has variables $x_i \in \{ \pm 1 \}$ for $i = 1, ..., n$ and the energy term can be expressed as
$$E(x) = - \sum_\ell J_\ell \prod_{s \in \Omega_\ell} x_s$$
where $\Omega_\ell \subseteq [n]$ contains at most $k$ spin, and each spin interacts with at most $d$ other spins, i.e. the dependency neighborhood is local. 

It is not true that the Ising on graph is necessarily a sub-class of $(k,d)$-local Ising for $k, d$ constants, as the graph in the former can have large degree ($d \approx n$) on some vertices thus the dependency neighborhood is large. A lattice ising is equivalent to $(k=2, d=4)$-local Ising. 

**In these notes it is paramount to consider $(k,d)$-local Ising as that's the format we can optimize the most.**

## Metropolis-Hastings algorithm

This is a general framework to construct a MC whose stationary distribution is the desider target distribution $\pi$, might be a Boltzmann distribution $\pi_\beta$ or it might be something else. 

Let $\Omega$ be the state space and $\pi$ be a distribution on $\Omega$ with $\pi_x > 0$ for all $x$ (we want no singularities caused by $\pi_x^{-1}$). 

We start by defining a column-stochastic **proposal matrix** $A$ that is the probability of proposing a move from $x$ to $y$. For example, for an Ising model it could be
$$A(x, y) = A_{yx} = \begin{cases} \frac{1}{n+1}, & y = x \oplus 2^j, \quad j \in [n] \\ 0, & y = x \\ 0, & \text{otherwise} \end{cases}$$
which basically is the uniform single spin-flip model: $x \oplus 2^j$ means flip the $j$-th spin, and the probability of doing so is 1/number of spins. 

The single spin flip model is a great example: it is local (a constant amount of spin to flip at each move, not growing with $n$, lead to efficient optimizations later), and symmetric (implies reversibility). 

Given $y \sim A(x, \cdot)$ the move is accepted with probability
$$\alpha(x, y) = \min\left(1, \frac{\pi(y) A(y, x)}{\pi(x) A(x, y)} \right)$$
Therefore
$$\begin{align*}
P(x, y) & = A(x, y) \alpha(x, y), & x \neq y, \\
P(x, x) & = 1 - \sum_{y \neq x} A(x, y) \alpha(x, y),
\end{align*}$$
By construction $P$ is column stochastic, enforces detailed balance, and includes self loops (aperiodic). If $A$ is irreducible, which basically is the only thing you need to prove by yourself, the constructed $P$ is irreducible, reversible, aperiodic: _it has a unique stationary state reachable by any initial state_. 

In the _special case_ of the Boltzmann distribution, the acceptance probability depends _uniquely_ on the energy difference:
$$\alpha(x, y) = \min\left( 1, e^{-\beta(E(y) - E(x))} \cdot \frac{A(y, x)}{A(x, y)} \right)$$
In this case, one commonly chooses $A$ to be symmetric so to simply this to the _Metropolis rule_:
$$\alpha(x, y) = \min\left( 1, e^{-\beta(E(y) - E(x))} \right)$$
i.e. always accept energy lowering moves, and accept energy increasing moves with probability exponentially suppressed in $\beta$. 

**Important**: this method has avoided us to calculate the partition function, which is great because it is unfeasible in the worst case and an annoyance in the best. 

**Important**: the efficiency is governed by the mixing time: complex energy landscapes as low-temperature Ising, the acceptance probabilities becomes small and the spectral gap drops. The convergence might be very slow. 

## Python implementation